In [1]:
import torch
import torch.nn as nn
import math
import numpy as np

np.random.seed(42)

In [2]:
class RequestType:
    def __init__(self, request_type, bandwidth, service_rate, arrival_rate, source, sink, distribution, switch_rate=None):
        # distribution is 1x2 if elastic and 1x1 if static
        
        self.type = request_type
        self.bw = bandwidth
        self.service_rate = service_rate
        self.arrival_rate = arrival_rate
        self.source = source
        self.sink = sink
        self.distribution = distribution
        self.switch_rate = switch_rate

class Request:
    def __init__(self, request_type, service_time, arrival_time, source, sink, transfer_rate, distribution=None):
        self.type = request_type
        self.service_time = service_time
        self.arrival_time = arrival_time
        self.source = source
        self.sink = sink
        self.bw = transfer_rate
        self.request_type = request_type
        
        if request_type == "elastic":
            self.distribution = distribution
            self.scale_requests = []
            
    def add_scale_request(self, req): 
        # we store related scale requests for elastic requests
        # not used if static request
        self.scale_requests.append(req)
            
    def get_encoding(self, nodes_in_environment):
        # as per our notes, this SHOULD return 1x5 tensor,
        # but we have one hot encodings INSIDE this tensor,
        # so we will flatten this and return, so the size will be
        # larger than 1x5
        
        # nodes_in_environment is a list of all the nodes in our graph
        # eg ["a", "b", "c"]
        
        # request is [one hot source, one hot destination, bw, service time, one hot type]
                
        one_hot_source = nn.functional.one_hot(torch.tensor([nodes_in_environment.index(self.source)]), num_classes=len(nodes_in_environment)).flatten()
        one_hot_dest   = nn.functional.one_hot(torch.tensor([nodes_in_environment.index(self.sink)]), num_classes=len(nodes_in_environment)).flatten()
    
        if self.request_type == "static" or self.request_type == "scale":
            one_hot_type = torch.tensor([1, 0])
        elif self.request_type == "elastic":
            one_hot_type = torch.tensor([0, 1])
            
        encoding = torch.cat([one_hot_source, 
                             one_hot_dest,
                             torch.tensor([self.bw]), 
                             torch.tensor([self.service_time]),
                             one_hot_type])
        
        return encoding

In [3]:
class Link:
    def __init__(self, node_1, node_2, bw_capacity):
        self.serving_requests = []
        self.nodes = [node_1, node_2]
        self.total_bw = bw_capacity
        
    def reset(self):
        self.serving_requests = []
        
    def add_request(self, request_obj):
        self.serving_requests.append(request_obj)
        
    def remove_request(self, request_obj):
        self.serving_requests.remove(request_obj)
        
    def remaining_bw(self): 
        # subtracting bw being used from total bw capacity
        bw_being_used = 0
        for req in self.serving_requests:
            bw_being_used += req.bw
            
        return (self.total_bw - bw_being_used)

In [4]:
class Environment:
    # requests_in_service_encoder = nn.RNN(????, 7)
    
    def __init__(self, nodes, links, request_blueprints):
        """
        nodes: list of strings where each string is just a name or identifier of a node
        links: list of tuples where in tuple t, t[0] is first node, t[1] is another node, and t[2] is bw capacity of the link
        request_blueprints: list of DeploymentRequest objects
        """
        self.nodes = nodes
        self.links = {}
        self.request_history = []
        self.E_history = []
        self.past_distributions = []
        self.request_blueprints = request_blueprints
        self.last_time = 0
        self.episode_timesteps = 600
        
        for link in links:
            if link[0] not in self.nodes or link[1] not in self.nodes:
                raise Exception("Node in link " + str(link) + " doesn't exist")
            
            link_obj = Link(*link)

            self.links[link[0] + link[1]] = link_obj
            self.links[link[1] + link[0]] = link_obj
            
        self.request_list = self.create_requests()
        self.request_queue = iter(self.request_list)
            
    def add_request(self, request, path=None): # we want to add this request to a link or path
        # path: a list of nodes that the request traverses including source and sink
        # if no path is specified, path is assumed to be [req.source, req.sink]
        
        if path is not None: 
            nodes = [[path[i], path[i + 1]] for i in range(len(path) - 1)]
            for node_pair in nodes:
                env.links[node_pair[0] + node_pair[1]].add_request(request)
        
        else:
            self.links[request.source + request.sink].add_request(request)
        
        self.request_history.append(request)
        # print(self.links[request.source + request.sink])
    
    def reset(self):
        for link in self.links.values():
            link.reset()
        self.request_history = []
        self.E_history = []
        self.past_distributions = []
        self.last_time = 0
        self.request_list = env.create_requests()
        self.request_queue = iter(self.request_list)
        
        return env.get_encoding()
        
    def reward(self, request, decision):
        base_rate = 1         # 1 when static
        type_bonus = 0.9      # 0.9 when static
        if request.type == "elastic":
            base_rate = request.bw 
            type_bonus = 1.1                # 1.1 when elastic
            
        r = request.bw * base_rate * request.service_time * type_bonus
        
        # if remaining bandwidth on link < 0, very "bad" reward
        remaining_bw = self.links[request.source + request.sink].remaining_bw()
        if remaining_bw < 0:
            return (-r * 10)
        
        if decision == "accept":
            return r
        
        if decision == "reject":
            if request.type == "static":
                return 0
            elif request.type == "elastic":
                current_sum = torch.tensor([0, 0])
                for dist in self.past_distributions:
                    current_sum += dist
                average_past_distribution = current_sum / len(self.past_distributions)
                current_req_distribution = torch.tensor(request.distribution)

                return -1 * r * math.exp(-nn.functional.kl_div(average_past_distribution, current_req_distribution))

                
                """
                past_distributions = []
                for req in self.request_history:
                    if req.request_type == "elastic":
                        past_distributions.append(req.distribution)
                
                average_past_distribution = torch.mean(past_distributions, dim=1)
                current_req_distribution = torch.tensor(request.distribution)
                
                if bool(average_past_distribution[0] < current_req_distribution[0]):
                    return -1 * r * math.exp(-nn.functional.kl_div(average_past_distribution, current_req_distribution))
                else:
                    return 0
                """
                
    def next_req(self):
        return next(self.request_queue)
                
    def step(self, req, action):
        # what happens if we have two requests that come in on the same timestep but there is only enough bandwidth for one?
        # do we the decision on the second request with knowledge of the first request
        # essentially, after we accept the first request, will we submit an updated encoding of the network to the policy network?
 
        # actions is a Nx2 matrix where the first column in the request and second is the decision
        # decision is either "accept" or "reject"
        # this is given by our agent
                
        if action[0] > 0.5:
            # accept request
            paths = (env.search(req.source, req.sink, [], []))
            paths.sort(key=lambda x: len(x)) # sort by shortest path
            # select the path we are using
            path = paths[action[1:4].argmax()]
            
            self.add_request(req, path)
        
            reward = env.reward(req, "accept")
        elif action[0] < 0.5:
            # reject
            reward = env.reward(req, "reject")
        
        obs = env.get_encoding()
        
        done = req.arrival_time > 600
        info = None
        
        return obs, reward, done, info
        
    def update_requests(self, current_time):
        # here, we remove expired requests and update E_history based off of the request stats
        
        for request in self.request_history:
            if (request.arrival_time + request.service_time) > self.last_time and (request.arrival_time + request.service_time) < current_time:
                # request has expired, let's remove it from the links
                for link in self.links.values():
                    if request in link.serving_requests:
                        link.remove_request(request)
                        
                if request.type == "elastic":
                    time_on_higher_bw = 0
                    for scale_req in request.scale_requests:
                        time_on_higher_bw += scale_req.service_time

                    time_on_lower_bw = request.service_time - time_on_higher_bw

                    # calculate E[history]
                    request_time = np.array([time_on_lower_bw, time_on_higher_bw])
                    request_bw = request.bw
                    result = (request_time / request_time.sum()).dot(request_bw)
                    self.past_distributions.append(request_time / request_time.sum())
                    self.E_history.append(result)

    def get_encoding(self):
        links_processed = [] 
        # these will store links that we have already encoded so we don't encode them again
        
        current_encoding = []
        
        # h = torch.zeros(7) # assuming 7 for h0 size
        # last_out = None
        
        env_encoding = []
        
        next_req = self.next_req()
        while next_req.type == "scale":
            next_req = self.next_req()
            
        self.update_requests(next_req.arrival_time)
        
        for link in self.links.values():
            if link in links_processed:
                continue

                        
            # Commented because we don't want to encode any queue for phase 1
            
            # for req in link.serving_requests
                # request is [one hot source, one hot destination, bw, service time, one hot type]
                
                # one_hot_source = nn.functional.one_hot(torch.tensor([self.nodes.index(req.source)]), num_classes=len(self.nodes))
                # one_hot_dest   = nn.functional.one_hot(torch.tensor([self.nodes.index(req.sink)]), num_classes=len(self.nodes))

                # req_tensor = torch.Tensor([]) # mismatched dimensions??!
                # last_out, h = self.requests_in_service_encoder(req_tensor, h)

            # current_encoding.append(torch.cat(torch.Tensor([link.remaining_bw]), last_out))
            # torch.stack(current_encoding)
            
            # check implementation later
            
            env_encoding.append(link.remaining_bw())
            
            links_processed.append(link)
            
        return torch.tensor(env_encoding), torch.tensor(next_req.get_encoding(env.nodes)), next_req
    
    def create_requests(self):
        requests = []
        
        for request_type in self.request_blueprints:
            arrival_times = []
            service_times = []
            last_arrival = 0
        
            while last_arrival < self.episode_timesteps: # we want to generate requests till we reach episode end
                last_arrival += np.random.exponential(request_type.arrival_rate)
                arrival_times.append(last_arrival)
                                
            for _ in arrival_times:
                service_times.append(np.random.exponential(request_type.service_rate))
                
            for arrival_time, service_time in zip(arrival_times, service_times):
                # start creating requests
                
                new_request = Request(request_type.type, service_time, arrival_time, request_type.source, request_type.sink, request_type.bw[0], request_type.distribution)
                requests.append(new_request)
                
                if request_type.type == "elastic": 
                    # we will start with the first bandwidth element as starting bw
                    # WE ASSUME that bw[0] < bw[1]
                    timesteps_from_deployment = 0
                    current_bw = request_type.bw[0]
                    while timesteps_from_deployment < service_time:
                        if current_bw == request_type.bw[0]:
                            # we want to generate a scale request to increase bw
                            scale_bw = request_type.bw[1] - current_bw
                            scale_service_time = np.random.exponential(request_type.switch_rate[0])
                            scale_request = Request("scale", scale_service_time, \
                                                    last_arrival + timesteps_from_deployment, request_type.source, \
                                                   request_type.sink, scale_bw)
                            requests.append(scale_request)
                            new_request.add_scale_request(scale_request)
                            
                            timesteps_from_deployment += scale_service_time
                            current_bw = request_type.bw[0] + scale_bw # also equal to request_type.distribution[1]
                        elif current_bw == request_type.bw[1]:
                            # we want to go to lower bw and spend some time there
                            time_spent_on_lower_bw = np.random.exponential(request_type.switch_rate[1])
                            timesteps_from_deployment += time_spent_on_lower_bw
                            current_bw = request_type.bw[0]
                            
        # sort requests by arrival time
        requests.sort(key=lambda x: x.arrival_time)
        return requests
    
    def search(self, source, dest, visited_a, paths):
        visited_a.append(source)
        # print(visited_a)

        for link in set(env.links.values()):
            visited = visited_a.copy()
            if source in link.nodes:
                if dest in link.nodes:
                    visited.append(dest)
                    paths.append(visited)

                x = link.nodes.copy()
                x.remove(source)
                if x[0] not in visited:
                    self.search(x[0], dest, visited.copy(), paths)
        return paths

In [5]:
env = Environment(["a", "b", "c", "d", "e", "f"], [["a", "b", 10], ["a", "c", 10], ["b", "d", 10], \
                                                   ["c", "d", 20], ["c", "e", 10], ["d", "f", 10], \
                                                   ["e", "f", 10]], \
                  [RequestType("static", [2], 0.5, 0.75, "a", "b", [1]), \
                  RequestType("static", [8], 1, 1.5, "a", "b", [1]), \
                  RequestType("elastic", [4, 9], 1, 1.5, "a", "b", [0.8, 0.2], switch_rate=[0.08, 0.02]), \
                  RequestType("static", [1], 1, 1.5, "c", "d", [1]), \
                  RequestType("static", [7], 0.5, 0.75, "c", "d", [1]), \
                  RequestType("elastic", [3, 13], 2, 3, "c", "d", [0.9, 0.1], switch_rate=[0.09, 0.01]), \
                  RequestType("static", [3], 0.5, 0.75, "e", "f", [1]), \
                   RequestType("static", [6], 1, 1.5, "e", "f", [1]), \
                    RequestType("elastic", [5, 8], 2, 3, "e", "f", [0.7, 0.3], switch_rate=[0.07, 0.03])])


                # self, request_type, bandwidth, service_rate, arrival_rate, source, sink, distribution, switch_rate=None

In [6]:
def policy(env_encoding, next_req_encoding, next_req_obj):    
    # find all paths between source and sink
    paths = (env.search(next_req_obj.source, next_req_obj.sink, [], []))
    paths.sort(key=lambda x: len(x)) # sort by shortest path
    selection = 0
    for path in paths:
        # check if this path works
        works = True
        nodes = [[path[i], path[i + 1]] for i in range(len(path) - 1)]
        for node_pair in nodes:
            if env.links[node_pair[0] + node_pair[1]].remaining_bw() < next_req_obj.bw:
                works = False
                
        if works:
            selection = paths.index(path)
            selection_one_hot = nn.functional.one_hot(torch.tensor([selection]), num_classes=3).flatten()
            return torch.cat([torch.tensor([1]), selection_one_hot])
        
    return torch.cat([torch.tensor([0]), torch.tensor([0,0,0])])

In [7]:
env_encoding, next_req_encoding, next_req_obj = env.reset()
done = False

while not done:
    decision = policy(env_encoding, next_req_encoding, next_req_obj)
    
    obs, reward, done, info = env.step(next_req_obj, decision)
    env_encoding, next_req_encoding, next_req_obj = obs
    
    print(env_encoding)
    print(decision)
    # print(env.request_history)

<ipython-input-4-2295fd131b55>:202: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(env_encoding), torch.tensor(next_req.get_encoding(env.nodes)), next_req
/usr/local/lib/python3.9/site-packages/torch/nn/functional.py:2610: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


tensor([10, 10, 10, 20, 10, 10, 10])
tensor([1, 1, 0, 0])
tensor([10, 10, 10, 20, 10, 10, 10])
tensor([1, 1, 0, 0])
tensor([10, 10, 10, 20, 10, 10, 10])
tensor([1, 1, 0, 0])
tensor([10, 10, 10, 20, 10, 10,  7])
tensor([1, 1, 0, 0])
tensor([ 8, 10, 10, 20, 10, 10,  7])
tensor([1, 1, 0, 0])
tensor([ 8, 10, 10, 13, 10, 10,  7])
tensor([1, 1, 0, 0])
tensor([ 8, 10, 10, 13, 10, 10,  7])
tensor([1, 1, 0, 0])
tensor([ 8, 10, 10, 13, 10, 10,  5])
tensor([1, 1, 0, 0])
tensor([ 8, 10, 10, 20, 10, 10,  0])
tensor([1, 1, 0, 0])
tensor([ 8, 10, 10, 19, 10, 10,  0])
tensor([1, 1, 0, 0])
tensor([ 2, 10, 10, 20, 10, 10,  5])
tensor([1, 1, 0, 0])
tensor([ 2, 10, 10, 20, 10, 10,  0])
tensor([1, 1, 0, 0])
tensor([ 2, 10, 10, 20, 10, 10,  0])
tensor([1, 1, 0, 0])
tensor([ 0, 10, 10, 20, 10, 10,  0])
tensor([1, 1, 0, 0])
tensor([ 2,  8,  8, 18, 10, 10,  0])
tensor([1, 0, 1, 0])
tensor([ 2, 10, 10, 20, 10, 10,  5])
tensor([1, 1, 0, 0])
tensor([ 2, 10, 10, 20, 10, 10,  2])
tensor([1, 1, 0, 0])
tensor([ 2, 10

tensor([1, 1, 0, 0])
tensor([10, 10, 10, 15, 10, 10,  7])
tensor([1, 1, 0, 0])
tensor([10, 10, 10, 15, 10, 10,  1])
tensor([1, 1, 0, 0])
tensor([ 6, 10, 10, 15, 10, 10,  1])
tensor([1, 1, 0, 0])
tensor([ 4, 10, 10, 16, 10, 10,  4])
tensor([1, 1, 0, 0])
tensor([ 2, 10, 10, 16, 10, 10,  4])
tensor([1, 1, 0, 0])
tensor([ 0, 10, 10, 16, 10, 10,  4])
tensor([1, 1, 0, 0])
tensor([ 0, 10, 10, 11,  5,  5, 10])
tensor([1, 0, 1, 0])
tensor([ 4, 10, 10, 11,  5,  5, 10])
tensor([1, 1, 0, 0])
tensor([ 8, 10, 10, 11,  5,  5, 10])
tensor([1, 1, 0, 0])
tensor([ 4, 10, 10, 11,  5,  5, 10])
tensor([1, 1, 0, 0])
tensor([ 4, 10, 10, 11,  5,  5, 10])
tensor([1, 1, 0, 0])
tensor([ 8, 10, 10, 11,  5,  5, 10])
tensor([1, 1, 0, 0])
tensor([ 8, 10, 10, 11,  5,  5,  5])
tensor([1, 1, 0, 0])
tensor([ 6, 10, 10, 11,  5,  5,  5])
tensor([1, 1, 0, 0])
tensor([ 4, 10, 10, 11,  5,  5,  5])
tensor([1, 1, 0, 0])
tensor([ 4, 10, 10, 16, 10, 10,  5])
tensor([1, 1, 0, 0])
tensor([ 6, 10, 10, 13, 10, 10,  5])
tensor([1, 1, 

tensor([5, 7, 7, 8, 4, 4, 2])
tensor([1, 1, 0, 0])
tensor([5, 7, 7, 8, 4, 4, 2])
tensor([0, 0, 0, 0])
tensor([7, 7, 7, 8, 4, 4, 2])
tensor([0, 0, 0, 0])
tensor([3, 7, 7, 8, 4, 4, 2])
tensor([1, 1, 0, 0])
tensor([3, 7, 7, 8, 4, 4, 2])
tensor([0, 0, 0, 0])
tensor([3, 7, 7, 5, 1, 1, 5])
tensor([1, 0, 1, 0])
tensor([3, 7, 7, 4, 1, 1, 5])
tensor([1, 1, 0, 0])
tensor([3, 7, 7, 4, 1, 1, 5])
tensor([0, 0, 0, 0])
tensor([1, 7, 7, 4, 1, 1, 5])
tensor([1, 1, 0, 0])
tensor([7, 7, 7, 4, 1, 1, 5])
tensor([0, 0, 0, 0])
tensor([7, 7, 7, 4, 1, 1, 2])
tensor([1, 1, 0, 0])
tensor([7, 7, 7, 3, 1, 1, 2])
tensor([1, 1, 0, 0])
tensor([7, 7, 7, 3, 1, 1, 2])
tensor([0, 0, 0, 0])
tensor([7, 7, 7, 3, 1, 1, 2])
tensor([0, 0, 0, 0])
tensor([3, 7, 7, 3, 1, 1, 2])
tensor([1, 1, 0, 0])
tensor([3, 7, 7, 3, 1, 1, 2])
tensor([0, 0, 0, 0])
tensor([3, 7, 7, 4, 1, 1, 2])
tensor([0, 0, 0, 0])
tensor([3, 7, 7, 1, 1, 1, 2])
tensor([1, 1, 0, 0])
tensor([10, 10, 10,  4,  1,  1,  2])
tensor([0, 0, 0, 0])
tensor([ 2, 10, 10,  4, 

tensor([10, 10, 10, 19,  3,  3,  0])
tensor([0, 0, 0, 0])
tensor([10, 10, 10, 12,  3,  3,  0])
tensor([1, 1, 0, 0])
tensor([10, 10, 10, 16,  7,  7, 10])
tensor([1, 0, 1, 0])
tensor([ 8, 10, 10, 16,  7,  7, 10])
tensor([1, 1, 0, 0])
tensor([10, 10, 10,  9,  7,  7, 10])
tensor([1, 1, 0, 0])
tensor([10, 10, 10,  2,  7,  7, 10])
tensor([1, 1, 0, 0])
tensor([10, 10, 10,  5, 10, 10, 10])
tensor([1, 1, 0, 0])
tensor([10, 10, 10, 12, 10, 10, 10])
tensor([1, 0, 1, 0])
tensor([10, 10, 10,  5, 10, 10, 10])
tensor([1, 1, 0, 0])
tensor([ 3,  3,  3,  5, 10, 10, 10])
tensor([1, 0, 1, 0])
tensor([10, 10, 10,  5, 10, 10, 10])
tensor([1, 0, 0, 1])
tensor([ 3,  3,  3,  5, 10, 10, 10])
tensor([1, 0, 1, 0])
tensor([ 3,  3,  3,  5, 10, 10,  7])
tensor([1, 1, 0, 0])
tensor([ 1,  3,  3,  5, 10, 10,  7])
tensor([1, 1, 0, 0])
tensor([ 3,  3,  3, 12, 10, 10, 10])
tensor([0, 0, 0, 0])
tensor([ 3,  3,  3, 12, 10, 10, 10])
tensor([1, 1, 0, 0])
tensor([ 3,  3,  3, 13, 10, 10,  7])
tensor([1, 1, 0, 0])


KeyboardInterrupt: 

In [84]:
env.links

{'ab': <__main__.Link at 0x131ac7eb0>,
 'ba': <__main__.Link at 0x131ac7eb0>,
 'ac': <__main__.Link at 0x131ac7c10>,
 'ca': <__main__.Link at 0x131ac7c10>,
 'bd': <__main__.Link at 0x131ac7d00>,
 'db': <__main__.Link at 0x131ac7d00>,
 'cd': <__main__.Link at 0x131ac7c40>,
 'dc': <__main__.Link at 0x131ac7c40>,
 'ce': <__main__.Link at 0x131ac72e0>,
 'ec': <__main__.Link at 0x131ac72e0>,
 'df': <__main__.Link at 0x131ac7250>,
 'fd': <__main__.Link at 0x131ac7250>,
 'ef': <__main__.Link at 0x131ac7b20>,
 'fe': <__main__.Link at 0x131ac7b20>}

In [9]:
env.links # notice that there are pairs that point to the same link obj

{'ab': <__main__.Link at 0x1315e0fa0>,
 'ba': <__main__.Link at 0x1315e0fa0>,
 'ac': <__main__.Link at 0x1315e8040>,
 'ca': <__main__.Link at 0x1315e8040>,
 'bd': <__main__.Link at 0x1315e80a0>,
 'db': <__main__.Link at 0x1315e80a0>,
 'cd': <__main__.Link at 0x1315e8100>,
 'dc': <__main__.Link at 0x1315e8100>,
 'ce': <__main__.Link at 0x1315e8160>,
 'ec': <__main__.Link at 0x1315e8160>,
 'df': <__main__.Link at 0x1315e81c0>,
 'fd': <__main__.Link at 0x1315e81c0>,
 'ef': <__main__.Link at 0x1315e8fd0>,
 'fe': <__main__.Link at 0x1315e8fd0>}

In [10]:
env.get_encoding() # this should return the remaining bandwidth on all links

tensor([10, 10, 10, 20, 10, 10, 10])

In [11]:
env.links['ab'].serving_requests[0].get_encoding(env.nodes) # sample encoding of a request

IndexError: list index out of range

In [9]:
current_sum = torch.tensor([0, 0])
for dist in env.past_distributions:
    current_sum += dist
    
print(current_sum)

tensor([-18957.5333,  40325.5333], dtype=torch.float64)


In [8]:
a = 0

for x in env.past_distributions:
    a += x[0]
    
a

-18957.533292818243

In [21]:
a = 0

for x in env.request_history[404].scale_requests:
    a += x.service_time
    
print(a)
print(env.request_history[404].service_time)

0.6518222148831949
0.6699353421824163


In [43]:
for x in env.request_history:
    if x.type == "e"


static
elastic
static
static
static
static
static
elastic
elastic
static
static
elastic
static
static
static
static
static
static
elastic
static
static
static
static
static
elastic
static
static
elastic
static
static
static
static
static
static
elastic
static
static
static
static
static
static
elastic
static
elastic
elastic
static
elastic
static
static
static
static
static
elastic
static
elastic
static
elastic
static
static
elastic
static
static
static
static
static
elastic
elastic
static
static
static
static
static
static
static
static
static
static
static
elastic
static
static
static
elastic
static
static
static
elastic
static
static
static
static
static
static
static
static
static
static
static
static
static
elastic
static
static
static
static
elastic
static
static
elastic
static
static
static
static
static
static
static
static
static
static
static
static
elastic
static
static
elastic
static
elastic
elastic
static
static
elastic
static
static
static
static
static
static
static
stati